In [84]:
from collections import defaultdict
from datetime import datetime
import torch
import numpy as np
import pandas as pd

In [2]:
ml_20m_path = '../../../../datasets/ml-20m'

In [3]:
file_path = ml_20m_path + '/ratings.csv'
data = pd.read_csv(file_path, encoding='latin-1', sep=',', engine='python', index_col='userId')

In [4]:
data

,movieId,rating,timestamp
userId,,,
1,2,3.5,1112486027
1,29,3.5,1112484676
1,32,3.5,1112484819
1,47,3.5,1112484727
1,50,3.5,1112484580
...,...,...,...
138493,68954,4.5,1258126920
138493,69526,4.5,1259865108
138493,69644,3.0,1260209457


In [5]:
actions = data
actions = actions.groupby('movieId').filter(lambda  x: len(x) >= 5)
actions = actions.groupby('userId').filter(lambda  x: len(x) >= 5)

In [6]:
actions = actions.groupby('userId', group_keys=False).apply(lambda  x: x.sort_values('timestamp'))

In [7]:
actions

,movieId,rating,timestamp
userId,,,
1,924,3.5,1094785598
1,919,3.5,1094785621
1,2683,3.5,1094785650
1,1584,3.5,1094785656
1,1079,4.0,1094785665
...,...,...,...
138493,6534,3.0,1260209908
138493,53464,4.0,1260209920
138493,1275,3.0,1262378552


In [8]:
# data = data.groupby('userId')
min = 100000
max = 0
total = 0
for i in range(1, actions.index.max()):
    total += len(actions.loc[i])
    if len(data.loc[i]) < min:
        min = len(actions.loc[i])
    if len(data.loc[i]) > max:
        max = len(actions.loc[i])
min, max, total/actions.index.max()

(20, 8540, 144.2935816250641)

In [9]:
usermap = dict()
usernum = 0
itemmap = dict()
itemnum = 0
# reorder the userid and itemid (keep the same step with original SASRec code)
for _id, row in actions.iterrows():
    if _id in usermap:
        userid = usermap[_id]
    else:
        usernum += 1
        userid = usernum
        usermap[_id] = userid

    if row.movieId in itemmap:
        itemid = itemmap[row.movieId]
    else:
        itemnum += 1
        itemid = itemnum
        itemmap[row.movieId] = itemid

In [10]:
usermap.__len__(), itemmap.__len__()

(138493, 18345)

In [11]:
actions['movieId'] = actions['movieId'].map(itemmap)
actions.index = actions.index.map(usermap)

In [12]:
actions

,movieId,rating,timestamp
userId,,,
1,1,3.5,1094785598
1,2,3.5,1094785621
1,3,3.5,1094785650
1,4,3.5,1094785656
1,5,4.0,1094785665
...,...,...,...
138493,872,3.0,1260209908
138493,797,4.0,1260209920
138493,2336,3.0,1262378552


In [13]:
sas_data = actions.drop('rating', axis=1, inplace=False).drop('timestamp', axis=1, inplace=False)

In [14]:
with open('ml-20m.txt', 'w') as f:
    for _id in set(sas_data.index):
        for movie_id in sas_data.loc[_id].movieId:
            f.write('%d %d\n' % (_id, movie_id))

In [15]:
sas_data

,movieId
userId,
1,1
1,2
1,3
1,4
1,5
...,...
138493,872
138493,797
138493,2336


In [16]:
userMaxTime = actions.groupby('userId').timestamp.max()

In [17]:
userMaxTime

userId
1         1112486201
2          974821014
3          945176099
4          840879654
5          851617728
             ...    
138489    1352990179
138490     975545576
138491    1247183347
138492    1115351450
138493    1262378572
Name: timestamp, Length: 138493, dtype: int64

In [30]:
day = 86400
num_day = 14
userSplitTime = userMaxTime - num_day * day

In [31]:
actions

,movieId,rating,timestamp
userId,,,
1,1,3.5,1094785598
1,2,3.5,1094785621
1,3,3.5,1094785650
1,4,3.5,1094785656
1,5,4.0,1094785665
...,...,...,...
138493,872,3.0,1260209908
138493,797,4.0,1260209920
138493,2336,3.0,1262378552


In [32]:
userSplitTime

userId
1         1111276601
2          973611414
3          943966499
4          839670054
5          850408128
             ...    
138489    1351780579
138490     974335976
138491    1245973747
138492    1114141850
138493    1261168972
Name: timestamp, Length: 138493, dtype: int64

In [38]:
def filter_input(group):
    user_id = group.index[0]
    limit = userSplitTime.loc[user_id]
    return group[group['timestamp'] < limit]

def filter_target(group):
    user_id = group.index[0]
    limit = userSplitTime.loc[user_id]
    return group[group['timestamp'] >= limit]

In [65]:
input_data = actions.groupby('userId').apply(filter_input)
target_window = actions.groupby('userId').apply(filter_target)

In [66]:
input_data = input_data.reset_index(level=1, drop=True)

In [54]:
input_data

,movieId,rating,timestamp
userId,,,
1,1,3.5,1094785598
1,2,3.5,1094785621
1,3,3.5,1094785650
1,4,3.5,1094785656
1,5,4.0,1094785665
...,...,...,...
138493,1901,4.5,1260209726
138493,978,3.5,1260209807
138493,971,4.0,1260209811


In [67]:
target_window = target_window.reset_index(level=1, drop=True)

In [55]:
target_window

,movieId,rating,timestamp
userId,,,
1,47,3.5,1112484493
1,48,3.5,1112484525
1,49,3.5,1112484537
1,50,3.0,1112484548
1,51,4.0,1112484560
...,...,...,...
138492,2955,4.0,1115351448
138492,189,5.0,1115351450
138493,2336,3.0,1262378552


In [68]:
input_idx = set(input_data.index)
target_idx = set(target_window.index)
valid_userid = input_idx.intersection(target_idx)

In [70]:
len(valid_userid)

39019

In [72]:
valid_user = sorted(list(valid_userid))
valid_user

[1,
 11,
 17,
 18,
 24,
 34,
 35,
 40,
 41,
 43,
 46,
 47,
 48,
 51,
 56,
 58,
 60,
 61,
 65,
 82,
 89,
 90,
 91,
 93,
 96,
 100,
 103,
 107,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 140,
 144,
 153,
 154,
 156,
 162,
 167,
 170,
 186,
 189,
 205,
 206,
 208,
 209,
 213,
 215,
 218,
 219,
 220,
 222,
 223,
 228,
 233,
 238,
 239,
 241,
 247,
 248,
 258,
 267,
 268,
 269,
 271,
 275,
 278,
 279,
 283,
 290,
 294,
 304,
 306,
 309,
 312,
 313,
 314,
 317,
 318,
 334,
 335,
 337,
 339,
 342,
 343,
 344,
 347,
 348,
 349,
 351,
 359,
 361,
 367,
 370,
 372,
 375,
 376,
 378,
 379,
 380,
 383,
 386,
 387,
 388,
 389,
 390,
 394,
 409,
 412,
 413,
 415,
 419,
 421,
 423,
 425,
 431,
 439,
 440,
 441,
 445,
 448,
 449,
 451,
 453,
 455,
 456,
 459,
 466,
 468,
 469,
 470,
 471,
 474,
 485,
 486,
 487,
 488,
 489,
 490,
 495,
 503,
 505,
 506,
 508,
 514,
 516,
 520,
 523,
 534,
 535,
 539,
 540,
 544,
 549,
 557,
 567,
 572,
 573,
 575,
 577,
 578,
 581,
 583,
 584,
 586,
 587,


In [87]:
with open('ml-20m_train.txt', 'w') as f:
    for _id in valid_user:
        movie_id_data = input_data.loc[_id].movieId
        if isinstance(movie_id_data, pd.Series):
            for movie_id in movie_id_data:
                f.write('%d %d\n' % (_id, movie_id))
        else:  # it's a single value, not a list
            f.write('%d %d\n' % (_id, movie_id_data))

In [83]:
list(input_data.loc[7448].movieId)

TypeError: 'numpy.float64' object is not iterable

In [88]:
with open('ml-20m_target.txt', 'w') as f:
    for _id in valid_user:
        movie_id_data = target_window.loc[_id].movieId
        if isinstance(movie_id_data, pd.Series):
            for movie_id in movie_id_data:
                f.write('%d %d\n' % (_id, movie_id))
        else:  # it's a single value, not a list
            f.write('%d %d\n' % (_id, movie_id_data))